In [1]:
import pickle
with open('coordinatemodel.pkl', 'rb') as file:  
    Pickled_Model= pickle.load(file)



In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

mpPose = mp.solutions.pose
mp_pose = mp.solutions.pose
pose = mpPose.Pose()
mp_drawing = mp.solutions.drawing_utils
points = mpPose.PoseLandmark 


In [3]:
data=list()
for p in points: #p will iterate through nose, eyes, ears etc...
        x = str(p)[13:]
        data.append(x + "_x") #will append nose_x
        data.append(x + "_y") #will append nose_y
        data.append(x + "_z") #will append nose_z
        data.append(x + "_vis") #will append nose_vis

dft = pd.DataFrame (columns=data)
num_images=0

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #ON: c[1] is y coordinate of c and b[1] is the y coordinate of b. that way we calculate the angle in radians
    #ON: then convert radians to angle and then do 360-angle if it is greater than 180 since hand or anything wont move 360 degree.
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
    #if angle<0:
        #angle+=360
        
    return angle 

In [6]:
from time import time
cap = cv2.VideoCapture(0)
two=[]
# Curl counter variables
counter = 0 
stage = None
time1 = 0
ti=0
msg=""
pred="no pose"
cv2.namedWindow('Pose Classification', cv2.WINDOW_NORMAL)

## Setup mediapipe instance
with mpPose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ok, frame = cap.read()
        if not ok:
        #continue to the next iteration to read the next frame and ignore the empty camera frame.
            continue
        frame = cv2.flip(frame, 1)
    
        # Get the width and height of the frame
        frame_height, frame_width, _ =  frame.shape
    
        # Resize the frame while keeping the aspect ratio.
        frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
   
      
        two=[]
        results = pose.process(image)
        if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
                landmarks = results.pose_landmarks.landmark
                for i,j in zip(points,landmarks):
                            two = two + [j.x, j.y, j.z, j.visibility]

        time2 = time()
        if (time2 - time1) > 0:
    
        # Calculate the number of frames per second.
            frames_per_second = 1.0 / (time2 - time1)
            cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
        try:
            time1 = time2
            dft.loc[1]=two
            pred = Pickled_Model.predict(dft)
            res=pred[0]
            #print(two)
        #print(pred)

        except:
            pass
                         
    
    
       
        
        if ti==0:
            cur_pose=pred
            ti=ti+1
        elif ti<30:
            if ti%15==0:
                leftElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                  [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                   [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
                rightElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])
                leftShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
                rightShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])
                leftKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
                rightKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])
                rightFoot=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y])
                leftFoot=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y])
                rightHip=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y])
                leftHip=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
                rightWrist=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])
                leftWrist=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
                           [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y])
                

                cur_pose=cur_pose
                ti=ti+1
                if cur_pose=="warrior2":
                    #also known as Virabhadrasana II
                    # Check if the both arms are straight.
                    if leftElbow > 165 and leftElbow < 195 and rightElbow > 165 and rightElbow < 195:
                        #  Check if shoulders are at the required angle.
                        if leftShoulder > 75 and leftShoulder < 110 and rightShoulder > 75 and rightShoulder< 110:
                            if leftKnee> 165 and leftKnee < 195 or rightKnee > 165 and rightKnee<195:  
                                # Check if the other leg is bended at the required angle.
                                if leftKnee> 90 and leftKnee< 150 or rightKnee> 90 and rightKnee < 150:
                                    msg="Kudos ! You are doing Warrior 2 pose perfectly.."
                                else:
                                    msg="Oops you are not holding your legs properly.."
                            else:
                                msg="Adjust your legs .."
                        else:
                            msg="Keep your shoulders straight .."
                    else:
                        msg="Adjust your hands.."
                    
                    
                    
                elif cur_pose=="no pose":
                    msg=" "
                    
                    
                elif cur_pose=="tree":
                    #also known as  Vrikshasana
                    #check if hands are folded
                    if leftElbow>65 and leftElbow < 180 and rightElbow>65 and rightElbow<180:
                            if leftShoulder>120 and leftShoulder<190 and rightShoulder>120 and rightShoulder<190:    
                                # Check if one leg is straight
                                if leftKnee>110 and leftKnee < 190 or rightKnee> 110 and rightKnee < 190:
                                    if rightKnee>10 and rightKnee<190 or leftKnee>10 and leftKnee<190:
                                        msg="You are doing tree pose well .. Superb !"
                                    else:
                                        print(leftKnee,"-l kn ",rightKnee," rkne")
                                        msg="Bend the leg properly.."
                                else:
                                    msg="Adjust your legs "
                            else:
                                msg=" "
                    else:
                        msg=" "
                  
                
                elif cur_pose=="downdog":
                    if leftElbow > 120 and leftElbow < 195 and rightElbow > 120 and rightElbow < 195:
                        if leftShoulder>150 and leftShoulder<180 or rightShoulder>150 and rightShoulder>180:
                            if rightHip>20 and rightHip<120 or leftHip>20 and leftHip<120:
                                    msg="Great ! You are doing downdog pose well.."
                            else:
                                msg="Push your hips higher up.."
                        else:
                            msg="Look back at your feet keeping your hands straight .."
                    else:
                        msg="Adjust your position"
                
                
                
                
                elif cur_pose=="plank":
                    #check if hands are straight
                    if leftKnee > 120 and leftKnee <180 or rightKnee >120 and rightKnee<180:
                        if leftElbow > 160 and leftElbow < 195 and rightElbow > 160 and rightElbow < 195:
                            if leftHip>140 and leftHip<195 or rightHip>140 or rightHip<195:
                                msg="Kudos ! Your plank is great !"
                            else:
                                msg="Cmon, you can do better. Try keeping your neck in line with your spine."
                        else:
                            msg="Keep your shoulders directly over your wrists.."
                    else:
                        msg="You are almost there.. Hold your legs straight .."

                
                
                
                elif cur_pose=="goddess":
                    if leftElbow > 60 and leftElbow < 130 and rightElbow > 60 and rightElbow < 130:
                        if leftKnee>60 and leftKnee<130 and rightKnee>60 and rightKnee<130:
                            if leftHip>60 and leftHip<130 and rightHip>60 and rightHip<130:
                                msg="Thats a perfect goddess pose ! "
                            else:
                                msg=" "
                        else:
                            msg="Keep a good amount of distance between your feet.. !"
                    else:
                        msg="Spread your fingers wide and extend your arms at shoulder height.."
                        
                
                
                elif cur_pose=="cobra":
                    if rightKnee>=140 and rightKnee<=180 or leftKnee>=140 and leftKnee<=180:
                        if leftHip>=90 and leftHip<=150 or rightHip>=90 and rightHip<=150:
                            msg="You are doing the pose well.. Good job !"
                        else:
                            msg="Bend your body properly.. "
                    else:
                        msg="Keep your legs straight .. "
                    
            else:
                ti=ti+1
        elif ti==30:
            ti=0
        
        
        # Setup status box
        cv2.rectangle(image, (0,0), (900,60), (255,153,153), -1)
        
        cv2.rectangle(image, (0,600), (900,700), (255,204,204), -1) #first parameter is the image
        #second parameter is the starting coordinate
        #third parameter is the bottom right co
        
        # Rep data
        cv2.putText(image, str(msg), (0,623),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (25,0,51), 1, cv2.LINE_AA) #first number moves the msg more towards right and second number moves downwards
                     #first number changes the size of msg and last determines thickness
        #the (0,0,0) value represents the colour of the text
        cv2.putText(image, str(res),(10,40),cv2.FONT_HERSHEY_SIMPLEX, 2, (25,0,51), 2, cv2.LINE_AA)
        
        
        #christy code
        if res=="tree":
            cv2.putText(image, str(""" Brings balance and equilibrium to mind.""" ), (70,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Helps improve concentration. """ ), (70,450), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Makes the legs stronger and improves balance.""" ), (70,500), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        elif res=="goddess":
            cv2.putText(image, str(""" Opens the hips and chest.""" ), (70,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Strengthens and tones the lower body """ ), (70,450), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Stimulates the respiratory and cardiovascular system.""" ), (70,500), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            
        elif res=="warrior2":
            cv2.putText(image, str(""" Opens chest and lungs and energizes tired limbs.""" ), (70,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Energizes tired limbs """ ), (70,450), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Stimulates abdominal organs""" ), (70,500), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Develops balance and stability""" ), (70,550), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Improves circulation and respiration""" ), (70,600), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
    
        elif res=="cobra":
            cv2.putText(image, str(""" Reduce symptoms of depression""" ), (70,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Relieve lower back pain """ ), (70,450), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Reduce inflammation""" ), (70,500), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Improve sleep""" ), (70,550), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        
        
        elif res=="downdog":
            cv2.putText(image, str(""" Stretches the lower body & strengthens the upper body.""" ), (70,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Stimulates blood flow """ ), (70,450), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Improves posture and fine-tunes the foot muscles""" ), (70,500), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
           
        elif res=="plank":
            cv2.putText(image, str(""" Reduces belly fat""" ), (70,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Improves body flexibility """ ), (70,450), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Improves metabolism""" ), (70,500), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Strengthens core""" ), (70,550), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(""" Reduces backache""" ), (70,600), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        #christy code end
    

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mpPose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 ) 
        cv2.imshow('Pose Classification', image)    
        
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
    
        

        
        
        
    cap.release()
    cv2.destroyAllWindows()